# Problem 2: Y-Chromosome Threshold Analysis - Implementation

**Objective**: Analyze the relationship between maternal BMI and timing of Y-chromosome concentration reaching ≥4% threshold using interval-censored survival analysis.

**Key Approach**: Use Accelerated Failure Time (AFT) models with proper interval censoring to handle multiple tests per mother and determine optimal testing weeks.

## 📋 Implementation Plan

This notebook implements the complete analysis following the detailed implementation guide:

1. **Section 1**: Event Interval Construction (Per Mother)
2. **Section 2**: Feature Matrix Preparation
3. **Section 3**: AFT Modeling & Inference (Core Analysis)
4. **Section 4**: Turnbull Non-parametric Validation
5. **Section 5**: BMI Grouping & Group-specific Optimal Weeks
6. **Section 6**: Monte Carlo Measurement Error Testing
7. **Section 7**: Baseline Two-step ML Comparison (Optional)
8. **Section 8**: Validation & Final Policy Table


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import sys
import re
from scipy import stats

# Core survival analysis
from lifelines import WeibullAFTFitter, LogLogisticAFTFitter, KaplanMeierFitter

# Machine learning
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score

# Add project root to Python path for imports
PROJECT_ROOT = Path("/home/richard/projects/cumcm")
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Import our custom modules
from src.analysis.problem2 import (
    apply_qc_filters,
    construct_intervals,
    prepare_feature_matrix,
    perform_group_specific_analysis,
    create_group_optimal_weeks_summary,
    run_monte_carlo_robustness_test,
    create_monte_carlo_summary_table,
    run_ml_baseline_comparison,
    run_comprehensive_validation
)
from src.models.aft_models import (
    fit_aft_models,
    display_model_summary,
    predict_survival_curves,
    calculate_optimal_weeks,
    fit_turnbull_estimator,
    compare_aft_vs_turnbull,
    assess_aft_goodness_of_fit
)

# Configure environment
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

print("✅ Libraries imported successfully")
print("✅ Survival analysis tools ready")
print("✅ Machine learning tools ready")


✅ Libraries imported successfully
✅ Survival analysis tools ready
✅ Machine learning tools ready


In [2]:
# Setup paths
DATA_PATH = PROJECT_ROOT / "src" / "data" / "data.xlsx"
OUTPUT_PATH = PROJECT_ROOT / "output"
OUTPUT_DATA_PATH = OUTPUT_PATH / "data"
OUTPUT_FIGURES_PATH = OUTPUT_PATH / "figures"
OUTPUT_RESULTS_PATH = OUTPUT_PATH / "results"

# Create output directories
OUTPUT_DATA_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_FIGURES_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_RESULTS_PATH.mkdir(parents=True, exist_ok=True)

print(f"✅ Paths configured - Data: {DATA_PATH}")
print(f"✅ Output paths ready")

# Load original male fetus data for interval construction
print("📂 Loading original male fetus data...")
male_data = pd.read_excel(DATA_PATH, sheet_name='男胎检测数据')
print(f"✅ Loaded {len(male_data)} rows and {len(male_data.columns)} columns")

print("\n📊 Data structure preview:")
print(f"  Columns: {list(male_data.columns[:10])}...")
print(f"  Maternal IDs: {male_data['孕妇代码'].nunique()} unique mothers")
print(f"  Total tests: {len(male_data)} test records")


✅ Paths configured - Data: /home/richard/projects/cumcm/src/data/data.xlsx
✅ Output paths ready
📂 Loading original male fetus data...
✅ Loaded 1082 rows and 31 columns

📊 Data structure preview:
  Columns: ['序号', '孕妇代码', '年龄', '身高', '体重', '末次月经', 'IVF妊娠', '检测日期', '检测抽血次数', '检测孕周']...
  Maternal IDs: 267 unique mothers
  Total tests: 1082 test records


## 📍 Section 1: Event Interval Construction (Per Mother)

**Goal**: Convert multiple test records per mother into single interval-censored event records.

**Event Definition**: Y-chromosome concentration first reaches ≥4% threshold

**Censoring Types**:
1. **Left-censored**: First observation already ≥4% → Event occurred before first test
2. **Interval-censored**: Threshold crossed between visits → Event in interval (L, R]  
3. **Right-censored**: Never reached threshold → Event beyond last observation


In [3]:
### Step 1.1: Data Preparation and QC Filtering

# Apply same QC as preprocessing to get df with individual tests
print("🔧 Applying QC filters from preprocessing...")
df_tests, filter_stats = apply_qc_filters(male_data, verbose=True)

print(f"\n📊 Individual test dataset ready:")
df_tests.head(10)


🔧 Applying QC filters from preprocessing...
🔧 Parsing variables and applying QC filters...
✅ Variable parsing completed:
  Gestational weeks: 1082/1082 valid
  BMI: 1082/1082 valid
  Y concentration: 1082/1082 valid
  After gestational weeks filter (10-25): 1069 records
  After GC content filter (40-60%): 620 records
  After chromosome abnormality filter: 556 records
  After missing data filter: 556 records
🎯 Applying IQR outlier detection...
  gestational_weeks:
    IQR outliers: 0 (0.00%) - bounds: [3.500, 29.214]
    After IQR filtering: 556 records (removed 0)
  bmi:
    IQR outliers: 13 (2.34%) - bounds: [24.982, 39.214]
    After IQR filtering: 543 records (removed 13)
  y_concentration:
    IQR outliers: 5 (0.92%) - bounds: [-0.017, 0.168]
    After IQR filtering: 538 records (removed 5)

✅ QC filtering completed: 538 records remaining (49.7% retention)
  Unique mothers: 238
  Tests per mother: 2.3 average

📊 Individual test dataset ready:


,maternal_id,gestational_weeks,bmi,y_concentration
0,A002,13.857143,33.331832,0.059230
1,A003,13.000000,30.742188,0.065185
2,A003,20.285714,31.882812,0.052253
3,A004,11.000000,28.641243,0.049498
4,A004,15.857143,28.641243,0.066800
5,A004,23.571429,29.161993,0.082347
6,A005,15.285714,30.844444,0.081533
7,A006,12.142857,35.883634,0.069469
8,A007,12.285714,33.874064,0.021902
9,A007,16.000000,33.874064,0.038038


In [4]:
### Step 1.2: Interval Construction Logic

# Generate interval-censored observations using module function
df_intervals = construct_intervals(df_tests, threshold=0.04, verbose=True)


🔄 Constructing interval-censored observations...
✅ Interval construction completed: 238 mothers

📊 Censoring type distribution:
  left: 203 (85.3%)
  interval: 22 (9.2%)
  right: 13 (5.5%)


In [5]:
### Step 1.3: Output Analysis

print(f"\n📊 Sample intervals:")
print(df_intervals.head(10)[['maternal_id', 'bmi', 'L', 'R', 'censor_type', 'n_tests', 'max_y_concentration']])



📊 Sample intervals:
  maternal_id        bmi          L          R censor_type  n_tests  max_y_concentration
0        A002  33.331832   0.000000  13.857143        left        1             0.059230
1        A003  30.742188   0.000000  13.000000        left        2             0.065185
2        A004  28.641243   0.000000  11.000000        left        3             0.082347
3        A005  30.844444   0.000000  15.285714        left        1             0.081533
4        A006  35.883634   0.000000  12.142857        left        1             0.069469
5        A007  33.874064  23.714286        inf       right        4             0.038038
6        A008  29.136316   0.000000  13.000000        left        2             0.060891
7        A009  33.333333   0.000000  13.857143        left        3             0.159267
8        A010  33.333333  20.857143        inf       right        2             0.031946
9        A011  36.250470  16.142857  20.571429    interval        4             0.068983


## 📍 Section 2: Feature Matrix Preparation

**Goal**: Prepare covariate matrix for AFT modeling with proper standardization and quality checks.


In [6]:
### Step 2.1: Standardization and Feature Matrix Creation

# Create feature matrix using module function
df_X = prepare_feature_matrix(df_intervals, verbose=True)


🔧 Preparing feature matrix for AFT modeling...
✅ BMI standardization completed:
  Mean BMI: 31.88
  Std BMI: 2.59
  BMI range: 26.6 - 39.2
✅ All intervals are valid (L < R)

📊 Feature matrix summary:
  Observations: 238
  Features: bmi, bmi_z
  Interval bounds: L, R
  Censoring types: ['left' 'right' 'interval']

📈 Dataset statistics:
              bmi         bmi_z           L           R
count  238.000000  2.380000e+02  238.000000  238.000000
mean    31.879303 -5.747037e-16    2.337335         inf
std      2.587130  1.000000e+00    5.824886         NaN
min     26.619343 -2.033125e+00    0.000000   11.000000
25%     29.917696 -7.582176e-01    0.000000   12.571429
50%     31.632725 -9.530933e-02    0.000000   13.428571
75%     33.453780  6.085806e-01    0.000000   16.142857
max     39.159843  2.814138e+00   24.285714         inf

🔍 Quality checks:
  Missing values: 0 total
  Interval validity check:
    Finite intervals: 225
    Right-censored: 13
    Left-censored: 203

📊 Summary by c

In [7]:
### Step 2.2: Quality Checks

# Quality checks are included in the prepare_feature_matrix function above
print("📋 Feature matrix preparation completed with quality checks - ready for AFT modeling")


📋 Feature matrix preparation completed with quality checks - ready for AFT modeling


## 📍 Section 3: AFT Modeling & Inference (Core Analysis)

**Goal**: Fit Accelerated Failure Time models to interval-censored data and derive BMI-specific survival functions.

**Models**: 
- Primary: Weibull AFT (log T = β'x + σε, ε ~ Gumbel)
- Alternative: Log-logistic AFT (ε ~ Logistic)

**Key Output**: Survival functions S(t|x) for calculating optimal testing weeks


In [8]:
### Step 3.1: Model Fitting

# Fit AFT models using module function
primary_model, primary_name, all_models = fit_aft_models(df_X, formula='~ bmi_z', verbose=True)


⚙️ Fitting AFT models to interval-censored data...

🔵 Fitting Weibull AFT model...
✅ Weibull AFT model fitted successfully

🟠 Fitting Log-logistic AFT model...
✅ Log-logistic AFT model fitted successfully

🎯 Using Weibull AFT as primary model


In [9]:
### Step 3.2: Model Summary & Diagnostics

# Display model summary and diagnostics using module function
display_model_summary(primary_model, primary_name, verbose=True)


📊 Weibull AFT Model Summary:
                       coef  exp(coef)  se(coef)  coef lower 95%  coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%  cmp to          z             p    -log2(p)
param   covariate                                                                                                                                                      
lambda_ Intercept  2.018579   7.527618  0.146708        1.731035        2.306122             5.646497            10.035431     0.0  13.759117  4.489752e-43  140.676272
        bmi_z      0.167024   1.181783  0.072897        0.024149        0.309899             1.024443             1.363288     0.0   2.291235  2.194982e-02    5.509647
rho_    Intercept  0.122470   1.130286  0.179968       -0.230260        0.475201             0.794327             1.608338     0.0   0.680512  4.961806e-01    1.011063

📈 Model Parameters:
  ('lambda_', 'Intercept'): 2.0186
  ('lambda_', 'bmi_z'): 0.1670
  ('rho_', 'Intercept'): 0.1225

🎯 BMI Effec

In [10]:
### Step 3.3: Survival Function Prediction

# Generate survival curves using module function
time_grid = np.linspace(10, 25, 100)  # Gestational weeks 10-25
survival_curves = predict_survival_curves(
    primary_model, 
    df_intervals, 
    time_grid, 
    quartiles=[0.25, 0.5, 0.75], 
    verbose=True
)


📈 Generating survival curves for different BMI levels...

📊 BMI quartiles for analysis:
  Q25: 29.9
  Q50: 31.6
  Q75: 33.5

🔄 Computing survival functions...
  ✅ Q25: BMI 29.9 (z=-0.76)
  ✅ Q50: BMI 31.6 (z=-0.10)
  ✅ Q75: BMI 33.5 (z=0.61)

📊 Survival Probabilities at Key Weeks:
Week  BMI_Q25     BMI_Q50     BMI_Q75     
------------------------------------------
12    0.142       0.179       0.222       
14    0.099       0.130       0.167       
16    0.066       0.091       0.123       
18    0.045       0.065       0.091       
20    0.031       0.046       0.068       

✅ Survival function prediction completed for 3 BMI levels


In [11]:
### Step 3.4: Optimal Testing Week Calculation

# Calculate optimal testing weeks using module function
optimal_weeks_results = calculate_optimal_weeks(
    survival_curves, 
    confidence_levels=[0.90, 0.95], 
    verbose=True
)


🎯 Calculating optimal testing weeks for different confidence levels...

📊 Optimal weeks for 90% confidence level:
  (When ≥90% of mothers have reached Y≥4% threshold)
------------------------------------------------------------
  Q25: BMI 29.9 → Week 13.9
  Q50: BMI 31.6 → Week 15.6
  Q75: BMI 33.5 → Week 17.6

📊 Optimal weeks for 95% confidence level:
  (When ≥95% of mothers have reached Y≥4% threshold)
------------------------------------------------------------
  Q25: BMI 29.9 → Week 17.6
  Q50: BMI 31.6 → Week 19.7
  Q75: BMI 33.5 → Week 22.1

📋 Optimal Testing Weeks Summary:
BMI Group    BMI Value  90% Conf   95% Conf  
---------------------------------------------
Q25          29.9       13.9       17.6      
Q50          31.6       15.6       19.7      
Q75          33.5       17.6       22.1      

✅ Optimal testing week calculation completed


## 📍 Section 4: Turnbull Non-parametric Validation

**Goal**: Validate AFT assumptions using non-parametric Turnbull estimator.

### Overview
- Fit Turnbull estimator to same interval-censored data
- Compare Turnbull vs AFT survival curves  
- Assess goodness of fit for parametric assumptions


In [12]:
### Step 4.1: Turnbull Fitting

# Fit Turnbull non-parametric estimator using module function
turnbull_model = fit_turnbull_estimator(df_X, verbose=True)


🔵 Fitting Turnbull non-parametric estimator...
Iteration 0 
   delta log-likelihood: 113.6900846918
   log-like:             -153.399751
Iteration 1 
   delta log-likelihood: -101.1526863254
   log-like:             -132.911080
Iteration 2 
   delta log-likelihood: 2.9231947109
   log-like:             -129.987885
Iteration 3 
   delta log-likelihood: -1.0849443151
   log-like:             -126.598301
Iteration 4 
   delta log-likelihood: 1.0252113341
   log-like:             -125.573090
Iteration 5 
   delta log-likelihood: -0.0043090313
   log-like:             -124.052893
Iteration 6 
   delta log-likelihood: 0.6773956086
   log-like:             -122.989866
Iteration 7 
   delta log-likelihood: -0.1124386356
   log-like:             -122.198954
Iteration 8 
   delta log-likelihood: 0.4953306406
   log-like:             -121.587947
Iteration 9 
   delta log-likelihood: -0.1470470728
   log-like:             -121.106272
Iteration 10 
   delta log-likelihood: 0.3827816919
   log-like:

In [13]:
### Step 4.2: Model Comparison

# Compare AFT vs Turnbull using median BMI as representative
median_bmi = df_intervals['bmi'].median()
comparison_results = compare_aft_vs_turnbull(
    primary_model, 
    turnbull_model, 
    time_grid, 
    median_bmi, 
    df_intervals, 
    verbose=True
)


📊 Comparing AFT model vs Turnbull non-parametric estimator...
  Restricting comparison to clinically meaningful range: 12.0 - 24.7 weeks
  Using 84 time points (avoids unreliable early extrapolation)
✅ Model comparison completed

📈 Population-Level Survival Curve Comparison:
  Turnbull: Non-parametric estimate using all 238 observations
  AFT: Parametric prediction averaged across all observations

📊 Comparison Metrics:
  Mean Absolute Error: 0.0141
  Root Mean Square Error: 0.0186
  Maximum Absolute Error: 0.0559
  KS Statistic: 0.0559
  ✅ Excellent agreement (MAE < 0.05)

📊 Survival Probabilities at Key Weeks:
Week   Turnbull     AFT          Difference  
------------------------------------------------
12     0.195        0.184        0.011       
14     0.108        0.139        0.031       
16     0.108        0.100        0.008       
18     0.066        0.074        0.007       
20     0.049        0.054        0.005       


In [14]:
### Step 4.3: Goodness of Fit Assessment

# Assess overall AFT model adequacy using module function
fit_assessment = assess_aft_goodness_of_fit(comparison_results, verbose=True)


🎯 AFT Model Goodness of Fit Assessment:

📊 Fit Quality Metrics:
  MAE: 0.0141 (threshold: < 0.1)
  RMSE: 0.0186 (threshold: < 0.15)
  KS: 0.0559 (threshold: < 0.2)

✅ Criteria Assessment:
  MAE: ✅ PASS
  RMSE: ✅ PASS
  KS: ✅ PASS

🎯 Overall Assessment:
  Criteria met: 3/3
  Overall fit quality: Good
  ✅ Recommendation: AFT model provides adequate fit to data
     The parametric AFT assumptions appear reasonable.

📋 Validation Summary:
  • Turnbull provides the non-parametric gold standard
  • AFT model offers interpretable covariate effects
  • Model agreement suggests good parametric fit


## 📍 Section 5: BMI Grouping & Group-specific Optimal Weeks

**Goal**: Create BMI groups and calculate optimal testing weeks for each group.

### Overview
- Calculate predicted median times for CART-based grouping
- Apply multiple BMI grouping strategies (clinical, tertile, CART) 
- Evaluate grouping strategies using risk-based scoring
- Calculate group-specific optimal weeks for different confidence levels
- Generate final policy recommendations by BMI group


In [15]:
### Step 5.1: Comprehensive Group-specific Analysis

# Perform group-specific analysis including:
# - Predicted median time calculation
# - Multiple BMI grouping strategies
# - Grouping strategy evaluation
# - Group-specific optimal weeks calculation
group_analysis_results = perform_group_specific_analysis(
    df_intervals, 
    primary_model,
    confidence_levels=[0.90, 0.95],
    time_grid=time_grid,
    grouping_methods=['clinical', 'tertile', 'cart'],  # Include CART if data allows
    verbose=True
)


🎯 Performing group-specific optimal weeks analysis...
🔄 Calculating individual predicted median survival times...
✅ Predicted median times calculated: 238/238 valid

📊 Predicted Median Time Statistics:
  Mean: 5.52 weeks
  Std: 0.98 weeks
  Range: 3.88 - 8.71 weeks

📊 Evaluating 4 grouping strategies...
📊 Evaluating grouping strategy: bmi_group_cart
  Number of groups: 6
  Risk score: 11.522
  Weighted median: 5.522 weeks
  Complexity penalty: 6.000
  Between-group variance: 1.187
  Within-group variance: 0.083

  Group sizes:
    CART_G2: 58 (24.4%)
    CART_G5: 46 (19.3%)
    CART_G1: 38 (16.0%)
    CART_G3: 35 (14.7%)
    CART_G4: 31 (13.0%)
    CART_G6: 30 (12.6%)
📊 Evaluating grouping strategy: bmi_group_clinical
  Number of groups: 3
  Risk score: 8.522
  Weighted median: 5.522 weeks
  Complexity penalty: 3.000
  Between-group variance: 2.240
  Within-group variance: 0.227

  Group sizes:
    Obese I (30-35): 147 (61.8%)
    Overweight (25-30): 61 (25.6%)
    Obese II+ (≥35): 30 

In [16]:
### Step 5.2: Generate Group-specific Optimal Weeks Summary

# Create comprehensive summary table for group-specific recommendations
group_summary_table = create_group_optimal_weeks_summary(group_analysis_results, verbose=True)


📋 Group-Specific Optimal Testing Weeks Summary:
         BMI_Group  n_mothers  representative_BMI BMI_range optimal_week_90 optimal_week_95
   Obese I (30-35)        147           32.017138 30.0-34.9            15.9            20.2
Overweight (25-30)         61           29.136316 26.6-30.0            13.3            16.7
   Obese II+ (≥35)         30           36.326687 35.1-39.2            21.1           Never

📊 Summary Statistics:
  Total mothers: 238
  Number of BMI groups: 3
  Grouping method: clinical


## 📍 Section 6: Monte Carlo Measurement Error Testing

**Goal**: Assess robustness to Y-chromosome concentration measurement errors.

### Overview
- Model measurement error as Gaussian noise: y_observed = y_true + ε, ε ~ N(0,σ²)
- Run Monte Carlo simulations with noisy measurements
- Refit AFT models and recalculate group-specific optimal weeks
- Assess stability and provide uncertainty quantification
- Generate robustness summary with confidence intervals


In [17]:
### Step 6.1: Monte Carlo Robustness Testing

# Run Monte Carlo simulation to assess robustness to measurement errors
# Use smaller sample size for demonstration (increase for production)
print("🎲 Starting Monte Carlo robustness testing...")
print("⏱️  Note: Using reduced simulation count for demonstration")

mc_results = run_monte_carlo_robustness_test(
    df_tests,  # Original test data
    construct_intervals,  # Function to construct intervals
    fit_aft_models,  # Function to fit AFT models  
    perform_group_specific_analysis,  # Function for group analysis
    n_simulations=300,  # Production setting for smooth CI and stable assessment
    sigma_error=0.002,  # 0.2% absolute concentration error
    confidence_levels=[0.90, 0.95],
    random_seed=42,
    n_workers=1,  # Single-threaded for stability
    verbose=True
)


🎲 Starting Monte Carlo robustness testing...
⏱️  Note: Using reduced simulation count for demonstration
🎲 Running Monte Carlo robustness test...
  Simulations: 300
  Measurement error σ: 0.0020 (0.20%)
  Confidence levels: ['90%', '95%']
🔄 Running simulations (single-threaded)...
  Progress: 30/300 (10.0%)
  Progress: 60/300 (20.0%)
  Progress: 90/300 (30.0%)
  Progress: 120/300 (40.0%)
  Progress: 150/300 (50.0%)
  Progress: 180/300 (60.0%)
  Progress: 210/300 (70.0%)
  Progress: 240/300 (80.0%)
  Progress: 270/300 (90.0%)
  Progress: 300/300 (100.0%)
📊 Analyzing Monte Carlo results...
✅ Monte Carlo Analysis:
  Successful iterations: 300/300 (100.0%)

  📊 Group: Obese I (30-35)
    90% confidence:
      Mean: 15.90 ± 0.38 weeks
      Median: 15.91 weeks
      95% CI: [15.15, 16.52]
      CV: 0.024
    95% confidence:
      Mean: 20.03 ± 0.56 weeks
      Median: 20.00 weeks
      95% CI: [19.09, 21.21]
      CV: 0.028

  📊 Group: Overweight (25-30)
    90% confidence:
      Mean: 13.55

In [18]:
### Step 6.2: Robustness Summary & Uncertainty Quantification

# Generate comprehensive robustness summary table
mc_summary_table = create_monte_carlo_summary_table(mc_results, verbose=True)


📋 Monte Carlo Robustness Summary:
         BMI_Group optimal_week_90_mean optimal_week_90_std optimal_week_90_ci stability_90 optimal_week_95_mean optimal_week_95_std optimal_week_95_ci stability_95
   Obese I (30-35)                15.90               ±0.38     [15.15, 16.52]         Good                20.03               ±0.56     [19.09, 21.21]         Good
Overweight (25-30)                13.55               ±0.67     [12.12, 14.85]         Good                17.05               ±0.78     [15.45, 18.64]         Good
   Obese II+ (≥35)                20.27               ±1.08     [18.33, 22.42]         Poor                24.06               ±0.80     [22.18, 25.00]         Good
     High BMI (T3)                18.24               ±0.56     [16.89, 19.24]         Good                23.03               ±0.92     [20.98, 24.48]         Good
   Medium BMI (T2)                15.62               ±0.39     [14.92, 16.36]         Good                19.75               ±0.59     [18.

## 📍 Section 7: Baseline Two-step ML Comparison (Optional)

**Goal**: Compare AFT survival approach against traditional machine learning methods.

### Overview
- **Classification Component**: Binary outcome (ever reached 4% threshold)
- **Regression Component**: Time to threshold (interval midpoint approximation)  
- **Feature Engineering**: BMI + derived features (squared, log-transformed)
- **Model Comparison**: Logistic Regression, Random Forest for both components
- **Group Mapping**: Map ML predictions to group-level recommendations
- **Benchmarking**: Compare AFT vs ML optimal weeks side-by-side


In [19]:
### Step 7.1: Complete ML Baseline Comparison Pipeline

# Run comprehensive ML baseline comparison including:
# - Binary classification (ever reached threshold)
# - Time-to-event regression (interval midpoint approximation)
# - Group-level recommendation mapping
# - Side-by-side comparison with AFT results

print("🤖 Running ML baseline comparison against AFT survival analysis...")

ml_comparison_results = run_ml_baseline_comparison(
    df_X,  # Use feature matrix with standardized BMI (bmi_z)
    group_analysis_results,  # AFT group-specific results from Section 5
    confidence_levels=[0.90, 0.95],
    cv_folds=5,
    random_state=42,
    verbose=True
)


🤖 Running ML baseline comparison against AFT survival analysis...
🤖 Running ML baseline comparison pipeline...
🔧 Preparing dataset for ML baseline comparison...
✅ ML dataset prepared:
  Samples: 238
  Features: 4 (bmi, bmi_z, bmi_squared, bmi_log)
  Ever reached threshold: 0.945
  Mean time approximation: 8.66 weeks
🎯 Training classification models for binary outcome...

  🔹 Training Logistic Regression...
    AUC (CV): 0.657 ± 0.207
    AUC (full): 0.677
    Brier score: 0.051

  🔹 Training Random Forest...
    AUC (CV): 0.565 ± 0.229
    AUC (full): 0.998
    Brier score: 0.015

✅ Best classification model: Logistic Regression
  Cross-validation AUC: 0.657
📈 Training regression models for time to threshold...

  🔹 Training Random Forest...
    MAE (CV): 3.499 ± 0.280 weeks
    RMSE (CV): 5.323 ± 0.470 weeks
    MAE (full): 1.575 weeks
    R²: 0.677

✅ Best regression model: Random Forest
  Cross-validation MAE: 3.499 weeks
🔄 Mapping ML predictions to group-level recommendations...

 

## 📍 Section 8: Validation & Final Policy Table

**Goal**: Generate final recommendations with comprehensive validation and uncertainty quantification.

### Overview
- **Cross-validation**: K-fold validation for model robustness assessment
- **Sensitivity Analysis**: Test different confidence levels and model assumptions
- **Final Policy Table**: Comprehensive recommendations with uncertainty bounds
- **Results Export**: Save final policy table for clinical implementation
- **Quality Assurance**: Complete validation summary with confidence metrics


In [20]:
### Step 8.1: Comprehensive Validation & Final Policy Generation

# Run complete validation pipeline including:
# - K-fold cross-validation for model robustness
# - Sensitivity analysis across confidence levels
# - Final policy table with uncertainty quantification
# - Automatic export to CSV for clinical use

print("🏁 Running comprehensive validation and final policy generation...")

final_validation_results = run_comprehensive_validation(
    df_intervals,  # Interval-censored dataset
    group_analysis_results,  # AFT group-specific results
    construct_intervals,  # Function to construct intervals
    fit_aft_models,  # Function to fit AFT models
    perform_group_specific_analysis,  # Function for group analysis
    mc_results=mc_results,  # Monte Carlo robustness results
    k_folds=5,  # Cross-validation folds
    confidence_levels=[0.90, 0.95],
    export_results=True,  # Export to CSV
    output_path=None,  # Use default path
    verbose=True
)


🏁 Running comprehensive validation and final policy generation...
🏁 Running comprehensive validation and final policy generation...
🔄 Performing 5-fold cross-validation of AFT pipeline...

  📊 Fold 1/5
    Train: 190 intervals from 190 mothers
    Test: 48 intervals from 48 mothers
    ✅ Fold completed: 48 test predictions

  📊 Fold 2/5
    Train: 190 intervals from 190 mothers
    Test: 48 intervals from 48 mothers
    ✅ Fold completed: 48 test predictions

  📊 Fold 3/5
    Train: 190 intervals from 190 mothers
    Test: 48 intervals from 48 mothers
    ✅ Fold completed: 48 test predictions

  📊 Fold 4/5
    Train: 191 intervals from 191 mothers
    Test: 47 intervals from 47 mothers
    ✅ Fold completed: 47 test predictions

  📊 Fold 5/5
    Train: 191 intervals from 191 mothers
    Test: 47 intervals from 47 mothers
    ✅ Fold completed: 47 test predictions

📊 Cross-validation Summary:
  Successful folds: 5/5
🔬 Performing sensitivity analysis...

  🎯 Testing different confidence lev